In [1]:
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn
from sklearn.preprocessing import StandardScaler
# define a XGBoost classifier
import xgboost as xgb
import warnings
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingClassifier
from aif360.sklearn.datasets import fetch_compas
from aif360.sklearn.metrics import disparate_impact_ratio, consistency_score, generalized_entropy_error
from aif360.sklearn.detectors import bias_scan
from aif360.sklearn.inprocessing import AdversarialDebiasing
from aif360.datasets import BinaryLabelDataset, StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric
from xgboost import XGBClassifier
from skl2onnx import update_registered_converter
from skl2onnx.common._apply_operation import apply_identity
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
)
from skl2onnx import update_registered_converter
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
warnings.filterwarnings("ignore")  # Ignore runtime warnings
# Temporarily adjust pandas display settings for large DataFrames
pd.set_option('display.max_rows', 100)  # Ensure 100 rows can be displayed
pd.set_option('display.max_columns', None)  # Ensure all columns can be displayed
pd.set_option('display.width', None)  # Automatically adjust display width to terminal size
pd.set_option('display.max_colwidth', None)  # Ensure full width of column content is shown
pd.set_option('display.float_format', '{:.4f}'.format)  # Format the float numbers for better readability


pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'
pip install 'aif360[OptimalTransport]'
pip install 'aif360[FACTS]'


In [2]:
data = pd.read_csv('../data/investigation_train_large_checked_adjusted2.csv')

In [3]:
class ZeroFeaturesByKeywords(BaseEstimator, TransformerMixin):
    def __init__(self, non_fair_keywords=None, to_remove=None):
        # Initialize keywords and columns to remove
        self.non_fair_keywords = non_fair_keywords if non_fair_keywords else [
            "relatie_kind", "relatie_partner", "ontheffing", "belemmering", 
            "competentie", "persoon", "persoonlijke", "adres", 
            "typering_dagen_som", "Ja", "Nee"
        ]
        self.to_remove = to_remove if to_remove else [
            "persoon_leeftijd_bij_onderzoek", "persoon_geslacht_vrouw",
            "belemmering_financiele_problemen", "persoonlijke_eigenschappen_taaleis_voldaan",
            "relatie_kind_heeft_kinderen"
        ]
        self.zeroed_columns_ = []

    def fit(self, X, y=None):
        # Identify columns to be zeroed (not modified in-place)
        self.feature_names_in_ = list(X.columns)

        # Identify columns to be zeroed
        self.zeroed_columns_ = [
        col for col in X.columns 
        if any(col.startswith(keyword) for keyword in self.non_fair_keywords) and col not in self.to_remove
        ]
        return self

    def transform(self, X):
        # Apply zeroing transformation
        X_copy = X.copy()
        for col in self.zeroed_columns_:
            X_copy[col] = 0
        return X_copy


In [4]:
# Step 2: Define the target and features
y = data['checked']  # Assuming 'checked' is the target column
X = data.drop(['checked'], axis=1)

# Ensure features are float32
X = X.astype(np.float32)

# Step 3: Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
)
classifier = XGBClassifier(max_depth=25, n_estimators=200)

# Step 6: Create the pipeline
pipeline = Pipeline([
    ('zero_features', ZeroFeaturesByKeywords()),  # Zero features transformer
    ('scaling', StandardScaler()),                # Scaling step
    ('classification', classifier)               # Final classifier
])

# Step 7: Train the pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('zero_features',
                 ZeroFeaturesByKeywords(non_fair_keywords=['relatie_kind',
                                                           'relatie_partner',
                                                           'ontheffing',
                                                           'belemmering',
                                                           'competentie',
                                                           'persoon',
                                                           'persoonlijke',
                                                           'adres',
                                                           'typering_dagen_som',
                                                           'Ja', 'Nee'],
                                        to_remove=['persoon_leeftijd_bij_onderzoek',
                                                   'persoon_geslacht_vrouw',
                                                   'belemmering_financiele_problemen',
                                                   'persoonlijke_eigenschappen_taa...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=25, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=200, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [5]:

# Step 4: Define the custom shape calculator and converter
def zero_features_shape_calculator(operator):
    operator.outputs[0].type = operator.inputs[0].type

def zero_features_converter(scope, operator, container):
    input_name = operator.inputs[0].full_name
    output_name = operator.outputs[0].full_name
    apply_identity(scope, input_name, output_name, container)

# Step 5: Register the custom transformer
update_registered_converter(
    ZeroFeaturesByKeywords,
    "ZeroFeaturesByKeywords",
    zero_features_shape_calculator,
    zero_features_converter
)


In [6]:

update_registered_converter(
    XGBClassifier,  # The model class
    "XGBoostXGBClassifier",  # The operator name
    calculate_linear_classifier_output_shapes,  # Shape calculator
    convert_xgboost, 
    options={"nocl": [True, False], "zipmap": [True, False, "columns"]},
)

In [19]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from skl2onnx import update_registered_converter
from skl2onnx.common._apply_operation import apply_identity

# Step 1: Define the custom transformer
class ZeroFeaturesByKeywords(BaseEstimator, TransformerMixin):
    def __init__(self, non_fair_keywords=None, to_remove=None):
        self.non_fair_keywords = non_fair_keywords if non_fair_keywords else [
            "relatie_kind", "relatie_partner", "ontheffing", "belemmering",
            "competentie", "persoon", "persoonlijke", "adres",
            "typering_dagen_som", "Ja", "Nee"
        ]
        self.to_remove = to_remove if to_remove else [
            "persoon_leeftijd_bij_onderzoek", "persoon_geslacht_vrouw",
            "belemmering_financiele_problemen", "persoonlijke_eigenschappen_taaleis_voldaan",
            "relatie_kind_heeft_kinderen"
        ]
        self.zeroed_columns_ = []

    def fit(self, X, y=None):
        self.feature_names_in_ = list(X.columns)
        self.zeroed_columns_ = [
            col for col in X.columns
            if any(col.startswith(keyword) for keyword in self.non_fair_keywords) and col not in self.to_remove
        ]
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in self.zeroed_columns_:
            X_copy[col] = 0
        return X_copy

# Step 2: Prepare the dataset
# data = pd.DataFrame(...)  # Load your dataset here
y = data['checked']  # Assuming 'checked' is the target column
X = data.drop(['checked'], axis=1)

X = X.astype(np.float32)  # Ensure features are float32

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y
)

# Step 3: Create and train the pipeline
classifier = XGBClassifier(max_depth=25, n_estimators=200)

pipeline = Pipeline([
    ('zero_features', ZeroFeaturesByKeywords()),  # Zero features transformer
    ('scaling', StandardScaler()),                # Scaling step
    ('classification', classifier)               # Final classifier
])

pipeline.fit(X_train, y_train)

# Step 4: Define the custom shape calculator and converter
def zero_features_shape_calculator(operator):
    operator.outputs[0].type = operator.inputs[0].type

def zero_features_converter(scope, operator, container):
    input_name = operator.inputs[0].full_name
    output_name = operator.outputs[0].full_name
    apply_identity(scope, input_name, output_name, container)

# Step 5: Register the custom transformer
update_registered_converter(
    ZeroFeaturesByKeywords,
    "ZeroFeaturesByKeywords",
    zero_features_shape_calculator,
    zero_features_converter
)

# Step 6: Register XGBoost classifier (if not already registered)
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost

update_registered_converter(
    XGBClassifier,
    "XGBoostXGBClassifier",
    calculate_linear_classifier_output_shapes,
    convert_xgboost,
    options={"nocl": [True, False], "zipmap": [True, False, "columns"]}
)

# Step 7: Convert the pipeline to ONNX
onnx_model = convert_sklearn(
    pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset={'': 12, 'ai.onnx.ml': 3}
)

# Step 8: Save the ONNX model
import onnx
onnx.save(onnx_model, "modelXGBnormal.onnx")


In [7]:
# Let's convert the model to ONNX
onnx_model = convert_sklearn(
    pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset={'': 12, 'ai.onnx.ml': 3})

#onnx.save(onnx_model, "modelXGBnormal.onnx")
testing_session = rt.InferenceSession("modelXGBnormal.onnx")

MissingShapeCalculator: Unable to find a shape calculator for type '<class '__main__.ZeroFeaturesByKeywords'>'.
It usually means the pipeline being converted contains a
transformer or a predictor with no corresponding converter
implemented in sklearn-onnx. If the converted is implemented
in another library, you need to register
the converted so that it can be used by sklearn-onnx (function
update_registered_converter). If the model is not yet covered
by sklearn-onnx, you may raise an issue to
https://github.com/onnx/sklearn-onnx/issues
to get the converter implemented or even contribute to the
project. If the model is a custom model, a new converter must
be implemented. Examples can be found in the gallery.


In [11]:
def calculate_prediction_consistency(testing_session, X_test, feature_name, values):
    """
    Calculate the fraction of cases where predictions are the same for different values of a given feature.

    Parameters:
        testing_session (InferenceSession): The ONNX model inference session.
        X_test (pd.DataFrame): Test dataset.
        feature_name (str): The feature to modify.
        values (list): List of two values to compare for the feature.

    Returns:
        float: Fraction of cases where predictions are the same.
    """
    same_predictions_count = 0
    total_samples = len(X_test)

    # Iterate through each sample in the test set
    for _, row in X_test.iterrows():
        # Convert the row to a DataFrame to ensure it's a DataFrame object
        X_sample = pd.DataFrame(row).transpose()

        # Make predictions for the first value
        X_sample_1 = X_sample.copy()
        X_sample_1[feature_name] = values[0]
        y_pred_1 = testing_session.run(None, {'X': X_sample_1.values.astype(np.float32)})

        # Make predictions for the second value
        X_sample_2 = X_sample.copy()
        X_sample_2[feature_name] = values[1]
        y_pred_2 = testing_session.run(None, {'X': X_sample_2.values.astype(np.float32)})

        # Check if predictions are the same
        if y_pred_1 == y_pred_2:
            same_predictions_count += 1

    # Calculate the fraction of cases where predictions are the same
    return same_predictions_count / total_samples

In [12]:
age_fraction = calculate_prediction_consistency(
    testing_session=testing_session,
    X_test=X_test,
    feature_name='persoon_leeftijd_bij_onderzoek',
    values=[25, 65]
)
print("Fraction of cases where predictions are the same for age 25 and 65:", age_fraction)


gender_fraction = calculate_prediction_consistency(
    testing_session=testing_session,
    X_test=X_test,
    feature_name='persoon_geslacht_vrouw',
    values=[0.0, 1.0]
)
print("Fraction of cases where predictions are the same for men and women:", gender_fraction)

language_fraction = calculate_prediction_consistency(
    testing_session=testing_session,
    X_test=X_test,
    feature_name='persoonlijke_eigenschappen_spreektaal_anders',
    values=[0.0, 1.0]
)
print("Fraction of cases where predictions are the same for dutch speakers and non-dutch speakers:", language_fraction)

children_fraction = calculate_prediction_consistency(
    testing_session=testing_session,
    X_test=X_test,
    feature_name='relatie_kind_heeft_kinderen',
    values=[0.0, 1.0]
)
print("Fraction of cases where predictions are the same for people with or without children:", children_fraction)


Fraction of cases where predictions are the same for age 25 and 65: 3.8902937171756467e-05
Fraction of cases where predictions are the same for men and women: 0.13814432989690723
Fraction of cases where predictions are the same for dutch speakers and non-dutch speakers: 1.0
Fraction of cases where predictions are the same for people with or without children: 1.0


In [13]:
def test_model_for_groups(pipe, x_training, y_training, x_testing, y_testing, group_dicts):
    """
    Tests the model and calculates fairness metrics for multiple privileged/unprivileged group definitions.
    
    Args:
        pipe: The model pipeline to test.
        x_training: Training features.
        y_training: Training labels.
        x_testing: Testing features.
        y_testing: Testing labels.
        group_dicts: List of dictionaries defining privileged and unprivileged groups.
    
    Returns:
        pd.DataFrame: Dataframe with calculated metrics for each privileged/unprivileged group combination.
    """
    def intoBinary(data, attribute, split):
        """
        Converts a continuous attribute into binary based on a percentile split.
        """
        sorted_data = data.sort_values(by=attribute)
        split_index = int(len(sorted_data) * split)
        split_threshold = sorted_data.iloc[split_index][attribute]
        print(f"Boundary for {attribute}: {split_threshold}")
        binary_attribute = pd.Series(0, index=data.index)
        binary_attribute[sorted_data[attribute] > split_threshold] = 1
        data[attribute] = binary_attribute
        return data

    if isinstance(pipe, rt.InferenceSession):  # ONNX model
        print("Using ONNX model for predictions")
        predictions = pipe.run(None, {'X': x_testing.values.astype(np.float32)})
        y_pred = predictions[0].round().astype(int)  # Ensure predictions are integer class labels
    else:  # Assume scikit-learn pipeline
        print("Training the model")
        pipe.fit(x_training, y_training)
        y_pred = pipe.predict(x_testing)

    # Prepare a DataFrame with features and predictions
    data_frame = pd.concat([x_testing.reset_index(drop=True), pd.DataFrame(y_pred, columns=['checked'])], axis=1)

    results = []

    for group in group_dicts:
        protected_attr = group['protected_attribute']
        privileged_groups = group['privileged']
        unprivileged_groups = group['unprivileged']
        
        # Convert the protected attribute to binary
        data_frame = intoBinary(data_frame, protected_attr, split=0.1)  # Using 0.1 as an example split value

        # BinaryLabelDataset creation
        bld = BinaryLabelDataset(
            df=data_frame,
            favorable_label=0,
            unfavorable_label=1,
            label_names=['checked'],
            unprivileged_protected_attributes=[[group['unprivileged_value']]],
            privileged_protected_attributes=[[group['privileged_value']]],
            protected_attribute_names=[protected_attr]
        )
        # Count outcomes for privileged and unprivileged groups
        privileged_indices = bld.protected_attributes[:, 0] == group['privileged_value']
        unprivileged_indices = bld.protected_attributes[:, 0] == group['unprivileged_value']

        privileged_labels = bld.labels[privileged_indices]
        unprivileged_labels = bld.labels[unprivileged_indices]

        privileged_checked_count = (privileged_labels == bld.favorable_label).sum()
        privileged_unchecked_count = (privileged_labels == bld.unfavorable_label).sum()

        unprivileged_checked_count = (unprivileged_labels == bld.favorable_label).sum()
        unprivileged_unchecked_count = (unprivileged_labels == bld.unfavorable_label).sum()

        # Print the counts
        print(f"\nCounts for Protected Attribute '{protected_attr}':")
        print(f"Privileged Group ({privileged_groups}):")
        print(f"  Checked (Favorable Outcomes): {privileged_checked_count}")
        print(f"  Unchecked (Unfavorable Outcomes): {privileged_unchecked_count}")
        print(f"Unprivileged Group ({unprivileged_groups}):")
        print(f"  Checked (Favorable Outcomes): {unprivileged_checked_count}")
        print(f"  Unchecked (Unfavorable Outcomes): {unprivileged_unchecked_count}")
        # Validate the dataset
        bld.validate_dataset()

        # Metric calculation
        metric = BinaryLabelDatasetMetric(bld, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)


        
        # Metrics
        disparate_impact = metric.disparate_impact()
        stat_par_diff = metric.statistical_parity_difference()
        mean_difference = metric.mean_difference()
        consistency = metric.consistency(n_neighbors=3)

        # Print metrics
        print(f"\nMetrics for protected attribute '{protected_attr}':")
        print(f"Privileged Groups: {privileged_groups}")
        print(f"Unprivileged Groups: {unprivileged_groups}")
        print(f"Disparate Impact: {disparate_impact}")
        print(f"Statistical Parity Difference: {stat_par_diff}")
        print(f"Mean Difference: {mean_difference}")
        print(f"Consistency: {consistency}")

        # Store the results
        results.append({
            'Protected Attribute': protected_attr,
            'Privileged Groups': privileged_groups,
            'Unprivileged Groups': unprivileged_groups,
            'Disparate Impact': disparate_impact,
            'Statistical Parity Difference': stat_par_diff,
            'Mean Difference': mean_difference,
            'Consistency': consistency
        })

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)
    print("\nFairness Metrics Summary:")
    print(results_df)

    return results_df


In [14]:
group_dicts = [
    {
        'protected_attribute': 'persoon_leeftijd_bij_onderzoek',
        'privileged': [{'persoon_leeftijd_bij_onderzoek': 1}],
        'unprivileged': [{'persoon_leeftijd_bij_onderzoek': 0}],
        'privileged_value': 1,
        'unprivileged_value': 0
    },
    {
        'protected_attribute': 'persoon_geslacht_vrouw',
        'privileged': [{'persoon_geslacht_vrouw': 0}],
        'unprivileged': [{'persoon_geslacht_vrouw': 1}],
        'privileged_value': 0,
        'unprivileged_value': 1
    },
    {
        'protected_attribute': 'relatie_overig_kostendeler',
        'privileged': [{'relatie_overig_kostendeler': 0}],
        'unprivileged': [{'relatie_overig_kostendeler': 1}],
        'privileged_value': 0,
        'unprivileged_value': 1
    }
]
results_df = test_model_for_groups(
    pipe=session, 
    x_training=X_train, 
    y_training=y_train, 
    x_testing=X_test, 
    y_testing=y_test, 
    group_dicts=group_dicts
)

Using ONNX model for predictions
Boundary for persoon_leeftijd_bij_onderzoek: 35.0

Counts for Protected Attribute 'persoon_leeftijd_bij_onderzoek':
Privileged Group ([{'persoon_leeftijd_bij_onderzoek': 1}]):
  Checked (Favorable Outcomes): 22371
  Unchecked (Unfavorable Outcomes): 761
Unprivileged Group ([{'persoon_leeftijd_bij_onderzoek': 0}]):
  Checked (Favorable Outcomes): 2210
  Unchecked (Unfavorable Outcomes): 363

Metrics for protected attribute 'persoon_leeftijd_bij_onderzoek':
Privileged Groups: [{'persoon_leeftijd_bij_onderzoek': 1}]
Unprivileged Groups: [{'persoon_leeftijd_bij_onderzoek': 0}]
Disparate Impact: 0.8881376340472437
Statistical Parity Difference: -0.10818230108633542
Mean Difference: -0.10818230108633542
Consistency: [0.94527654]
Boundary for persoon_geslacht_vrouw: 0.0

Counts for Protected Attribute 'persoon_geslacht_vrouw':
Privileged Group ([{'persoon_geslacht_vrouw': 0}]):
  Checked (Favorable Outcomes): 12608
  Unchecked (Unfavorable Outcomes): 624
Unpri

In [15]:
def evaluate_model_by_group(testing_session, X_test, y_test, group_column, group_mapping=None, bins=None, labels=None):
    """
    Evaluate model performance across demographic groups.

    Parameters:
        testing_session (InferenceSession): The ONNX model inference session.
        X_test (pd.DataFrame): Test dataset features.
        y_test (pd.Series): True labels for the test dataset.
        group_column (str): Column used to define groups.
        group_mapping (dict): Mapping of values in `group_column` to group labels.
        bins (list): Bin edges for numeric grouping.
        labels (list): Labels for bins if numeric grouping is used.

    Returns:
        dict: A dictionary containing performance metrics for each group.
    """
    # Copy test data to avoid modification
    X_test_grouped = X_test.copy()

    # Handle group definitions (mapping or binning)
    if group_mapping:
        X_test_grouped['group'] = X_test_grouped[group_column].map(group_mapping)
    elif bins and labels:
        X_test_grouped['group'] = pd.cut(X_test_grouped[group_column], bins=bins, labels=labels)
    else:
        raise ValueError("Provide either `group_mapping` or `bins` and `labels` to define groups.")

    # Drop rows with NaN in the 'group' column
    X_test_grouped = X_test_grouped.dropna(subset=['group'])

    # Align y_test with X_test_grouped
    y_test_aligned = y_test.loc[X_test_grouped.index]

    # Debug: Check group distribution
    print("Group distribution:\n", X_test_grouped['group'].value_counts())

    # Initialize dictionary to store results
    results = {}

    # Evaluate performance for each group
    for group in X_test_grouped['group'].unique():
        # Filter data for the current group
        X_group = X_test_grouped[X_test_grouped['group'] == group].drop(columns=['group'])
        y_group = y_test_aligned[X_test_grouped['group'] == group]

        # Predict using the model
        y_pred_group = testing_session.run(None, {'X': X_group.values.astype(np.float32)})[0]

        # Calculate evaluation metrics
        accuracy_group = accuracy_score(y_group, y_pred_group)
        precision_group = precision_score(y_group, y_pred_group, zero_division=0)
        recall_group = recall_score(y_group, y_pred_group, zero_division=0)
        f1_score_group = f1_score(y_group, y_pred_group, zero_division=0)

        # Store metrics for the group
        results[group] = {
            'Accuracy': accuracy_group,
            'Precision': precision_group,
            'Recall': recall_group,
            'F1 Score': f1_score_group
        }

    return results

In [16]:
age_bins = [0, 30, 40, 50, 60, 120]
age_labels = ['young_adult', 'youngish_adult', 'middle_aged_adult', 'older_adult', 'senior']

age_results = evaluate_model_by_group(
    testing_session=testing_session,
    X_test=X_test,
    y_test=y_test,
    group_column='persoon_leeftijd_bij_onderzoek',
    bins=age_bins,
    labels=age_labels
)

print("Age Group Results:")
for group, metrics in age_results.items():
    print(f"{group}: {metrics}")


gender_mapping = {1.0: 'woman', 0.0: 'man'}

gender_results = evaluate_model_by_group(
    testing_session=testing_session,
    X_test=X_test,
    y_test=y_test,
    group_column='persoon_geslacht_vrouw',
    group_mapping=gender_mapping
)

print("Gender Results:")
for group, metrics in gender_results.items():
    print(f"{group}: {metrics}")

language_mapping = {1.0: 'other_language', 0.0: 'not'}

language_results = evaluate_model_by_group(
    testing_session=testing_session,
    X_test=X_test,
    y_test=y_test,
    group_column='persoonlijke_eigenschappen_spreektaal_anders',
    group_mapping=language_mapping
)

print("Language Results:")
for group, metrics in language_results.items():
    print(f"{group}: {metrics}")

children_mapping = {1.0: 'has_child', 0.0: 'not'}

children_results = evaluate_model_by_group(
    testing_session=testing_session,
    X_test=X_test,
    y_test=y_test,
    group_column='relatie_kind_heeft_kinderen',
    group_mapping=children_mapping
)

print("Children Results:")
for group, metrics in children_results.items():
    print(f"{group}: {metrics}")

Group distribution:
 group
older_adult          8776
middle_aged_adult    8108
youngish_adult       4104
senior               3690
young_adult          1027
Name: count, dtype: int64
Age Group Results:
older_adult: {'Accuracy': 0.945077484047402, 'Precision': 0.825, 'Recall': 0.06496062992125984, 'F1 Score': 0.12043795620437957}
young_adult: {'Accuracy': 0.807205452775073, 'Precision': 0.8513513513513513, 'Recall': 0.41721854304635764, 'F1 Score': 0.56}
youngish_adult: {'Accuracy': 0.8135964912280702, 'Precision': 0.8755656108597285, 'Recall': 0.3527803099361896, 'F1 Score': 0.5029239766081871}
senior: {'Accuracy': 0.9, 'Precision': 0.8838174273858921, 'Recall': 0.3844765342960289, 'F1 Score': 0.5358490566037736}
middle_aged_adult: {'Accuracy': 0.8807350764676862, 'Precision': 0.849802371541502, 'Recall': 0.18793706293706294, 'F1 Score': 0.30780243378668576}
Group distribution:
 group
man      13232
woman    12473
Name: count, dtype: int64
Gender Results:
man: {'Accuracy': 0.8948004836

In [18]:
def calculate_fairness_stats(privileged_total, privileged_checked, unprivileged_total, unprivileged_checked):
    unprivileged_unchecked = unprivileged_total - unprivileged_checked
    privileged_unchecked = privileged_total - privileged_checked

    favorable_outcome_given_unprivileged = unprivileged_unchecked / unprivileged_total if unprivileged_total > 0 else 0
    favorable_outcome_given_privileged = privileged_unchecked / privileged_total if privileged_total > 0 else 0

    spd = favorable_outcome_given_unprivileged - favorable_outcome_given_privileged
    di = (favorable_outcome_given_unprivileged / favorable_outcome_given_privileged 
          if favorable_outcome_given_privileged > 0 else float('inf'))

    return {
        "Disparate Impact": di,
        "Statistical Parity Difference": spd
    }

# Helper function to evaluate predicates
def evaluate_predicate(df, predicate):
    return df.loc[predicate(df)], df.loc[~predicate(df)]

# Function to calculate and display fairness statistics for predicates
def calculate_and_display_fairness(df, predicates, checked_col):
    for i, predicate in enumerate(predicates):
        # Apply the predicate
        subset, subset_complement = evaluate_predicate(df, predicate)
        
        # Calculate counts
        total_count = subset.shape[0]
        checked_count = subset[subset[checked_col] == True].shape[0]
        total_count_complement = subset_complement.shape[0]
        checked_count_complement = subset_complement[subset_complement[checked_col] == True].shape[0]

        # Calculate fairness statistics
        fairness_stats = calculate_fairness_stats(
            total_count_complement, checked_count_complement, 
            total_count, checked_count
        )

        # Display results
        print(f"Stats for Predicate {i+1}:")
        for stat, value in fairness_stats.items():
            print(f"  {stat}: {value:.4f}")
        print("-" * 50)

# Reusable predicate functions
def is_in_age_range(min_age, max_age):
    return lambda df: (df['persoon_leeftijd_bij_onderzoek'] >= min_age) & (df['persoon_leeftijd_bij_onderzoek'] < max_age)

def is_female():
    return lambda df: df['persoon_geslacht_vrouw'] == True

def is_male():
    return lambda df: df['persoon_geslacht_vrouw'] == False

def has_children():
    return lambda df: df['relatie_kind_heeft_kinderen'] == True

def no_children():
    return lambda df: df['relatie_kind_heeft_kinderen'] == False

def lacks_language_skills():
    return lambda df: df['persoonlijke_eigenschappen_taaleis_voldaan'] == False

def has_language_skills():
    return lambda df: df['persoonlijke_eigenschappen_taaleis_voldaan'] == True

def lives_in_district(district):
    return lambda df: df[f'adres_recentste_wijk_{district}'] == True

def financial_difficulties():
    return lambda df: df['belemmering_financiele_problemen'] == True

def combine_and(*predicates):
    return lambda df: np.logical_and.reduce([predicate(df) for predicate in predicates])

def combine_or(*predicates):
    return lambda df: np.logical_or.reduce([predicate(df) for predicate in predicates])



session = rt.InferenceSession("modelXGBnormal.onnx")

# Prepare the input for the ONNX model
input_name = session.get_inputs()[0].name
X_test_np = X_test.to_numpy().astype('float32')

# Run predictions using the ONNX model
predictions = session.run(None, {input_name: X_test_np})[0]

# Add predictions to the dataframe
df = X_test.copy()  # Copy X_test to the new dataframe
df['checked'] = predictions  # Add predictions as the 'checked' column
# Define predicates
predicates = [
    combine_and(has_children(), lacks_language_skills()),
    combine_and(is_female(), financial_difficulties()),
    is_in_age_range(20, 35)
]

# Calculate and display fairness statistics
calculate_and_display_fairness(df, predicates, 'checked')

Stats for Predicate 1:
  Disparate Impact: 0.9815
  Statistical Parity Difference: -0.0177
--------------------------------------------------
Stats for Predicate 2:
  Disparate Impact: 0.9927
  Statistical Parity Difference: -0.0069
--------------------------------------------------
Stats for Predicate 3:
  Disparate Impact: 0.8889
  Statistical Parity Difference: -0.1073
--------------------------------------------------
